In [144]:
import requests
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import string
import spacy
from nltk.corpus import stopwords
from langdetect import detect
import re
from googletrans import Translator
#from google_trans_new import google_translator
import googletrans
trans = Translator()
from textblob import TextBlob
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/sinatxe/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
#driver = webdriver.Chrome(ChromeDriverManager().install(), options = opciones)
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False    # si True, no aperece la ventana (headless=no visible)

opciones.add_argument('--start-maximized')         # comienza maximizado

#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies

#opciones.add_extension('driver_folder/adblock.crx')       # adblocker

opciones.add_argument('--incognito') 

In [3]:
driver = "./chromedriver.exe"
driver = webdriver.Chrome(driver,options = opciones)

url2 = "http://estersinatxe.blogspot.com/"
driver.get(url2)
driver.implicitly_wait(2)
driver.find_element_by_css_selector("#cookieChoiceDismiss").click()

tit = []
texto = []

for i in range(1,8): 
    try:
        for p in range(1,8):
            tit.append(driver.find_element_by_css_selector(f"#Blog1 > div.blog-posts.hfeed > div:nth-child({p}) > div > div.post-outer > div > h3 > a").text)
            
            texto.append(driver.find_element_by_css_selector(f"#Blog1 > div.blog-posts.hfeed > div:nth-child({p}) > div > div.post-outer > div").text)
        driver.find_element_by_css_selector("#blog-pager-older-link").click()

    except:
        texto.append("no data")
driver.quit()

<ipython-input-3-00f320552e41>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver,options = opciones)
<ipython-input-3-00f320552e41>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector("#cookieChoiceDismiss").click()
<ipython-input-3-00f320552e41>:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  tit.append(driver.find_element_by_css_selector(f"#Blog1 > div.blog-posts.hfeed > div:nth-child({p}) > div > div.post-outer > div > h3 > a").text)
<ipython-input-3-00f320552e41>:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  texto.append(driver.find_element_by_css_selector(f"#Blog1 > div.blog-posts.hfeed > div:nth-child({p}) > div > div.post-outer > div").text)
<ipython-input-3-00f320552e41>:18: DeprecationWarning: find_element_

In [4]:
len(texto),len(tit)

(50, 49)

In [5]:
textos = []
for t in texto[:-1]:
    s = t.split("\n")
    ind = s.index("Enviar por correo electrónico")
    firma = s[1:ind-1] #auitamos el título
    if "Ester Sinatxe" in firma:
        inddos = firma.index("Ester Sinatxe")
        text_def = firma[1:inddos]
        textos.append(" ".join(text_def).strip())
    else:
        textos.append(" ".join(firma).strip())


In [6]:
textos[30]

'Hace cinco años, tres meses y diecisiete días que no cuento las horas que me quedan para vernos. Hace miles de hojas en blanco que no escribo porque no me das las ganas. Hace viento que se lleva las palabras.  Nos hicimos a imagen y semejanza de nuestro reflejo en los charcos. Nos deshicimos desesperando por los 5 minutos tarde que siempre llega la oportunidad.  Hubo un día en el que abrimos el libro por el final para leer un parasiempre que dejamos arder a 451 °F. Hubo un día en el que perdimos el tiempo apostando por la rutina. Hubo un día en el que congelamos las sobras de todo lo que nos hacía falta.  Ayer le dimos la espalda a la noche para no ver llegar el mañana.  Hoy celebro otro feliz no aniversario sin quererte ni pastas.   Ester Sinatxe.'

In [7]:
listando = []
for i in range(len(tit)):
    dicc= {}
    dicc["Titulo"] = tit[i]
    dicc["text"] = textos[i]
    listando.append(dicc)

In [8]:
textos_df = pd.DataFrame(listando)

In [9]:
textos_df.head(3)

,Titulo,text
0,"Cómo despedirse de alguien, a quien ya no salu...",A Patricia Primero: Dibújate un interrogante ...
1,Números rojos,Números rojos Subí la apuesta a que te bajaba...
2,En espiral,"Sufro una crisis de infertilidad, y ya no sé c..."


In [14]:

def english(col):
    try:
    
        trad = trans.translate(col,dest="en")
        return trad.text
    except:
        return col

In [15]:
textos_df["english"] = textos_df["text"].apply(english)

In [52]:
textos_df.rename(columns = {"Titulo":"tit","text":"tex","english":"tex_en"},inplace = True)

In [197]:
textos_df.head()

,tit,tex,tex_en,tok_en,sentiments,blobpol
0,"Cómo despedirse de alguien, a quien ya no salu...",A Patricia Primero: Dibújate un interrogante ...,A Patricia first: Draw a question on the lips ...,patricia draw question lip match white disbeli...,"[0.04540816326530611, 0.3739795918367347, 0.23...",0.045408
1,Números rojos,Números rojos Subí la apuesta a que te bajaba...,Red numbers I went up the bet that lowered you...,red number go bet lower pant end buy battery c...,"[0.09999999999999999, 0.38571428571428573, 0.1...",0.100000
2,En espiral,"Sufro una crisis de infertilidad, y ya no sé c...","I suffer a crisis of infertility, and I do not...",suffer crisis infertility know lethargy mascot...,"[-0.03958333333333334, 0.6124999999999999, 0.2...",-0.039583
3,En mis recuerdos,"En mis recuerdos siempre hay un cenicero, ropa...","In my memories there is always an ashtray, dir...",memory ashtray dirty clothe floor clock lose t...,"[-0.22999999999999998, 0.6399999999999999, 0.2...",-0.230000
4,Aquel rojo,Me he vuelto a malpintar las uñas de aquel roj...,I have returned to mischiece the nails of that...,return mischiece nail red combine underwear fi...,"[-0.16666666666666666, 0.5555555555555555, 0.0...",-0.166667


- instalar python -m spacy download es_core_news_sm
- instalar python -m spacy download en_core_news_sm


In [54]:
nlp = spacy.load("en_core_web_sm")
nlp_es = spacy.load("es_core_news_sm")

In [ ]:
# trans = Translator()
# esp = "que tengas un buen day"
# en = trans.translate(esp, dest="en")
# print(en.text)

# trans = Translator(service_urls=['translate.googleapis.com'])
# textil = textos_df.text[0]
# textil_en = trans.translate(textil,dest="en")
# print(textil_en.text)

In [55]:
stop_es = nlp_es.Defaults.stop_words
stop = nlp.Defaults.stop_words

In [ ]:
# tok = nlp_es(textil)
# filtro = []
# for t in tok:
#     if not t.is_stop:
#         lemma = t.lemma_.lower().strip()
#         if re.search('^[a-zA-Z]+$',lemma):
#             filtro.append(trans.translate(lemma,dest="en").text)
# textillo = " ".join(filtro)


In [56]:
def tokenizer (txt):    
    tokens = nlp(txt)
    filtradas = []
    for token in tokens:
        if not token.is_stop:
            lemma = token.lemma_.lower().strip()
            if re.search('^[a-zA-Z]+$',lemma): # Esto me quita las interrogaciones
                filtradas.append(lemma)
    return " ".join(filtradas)

In [202]:
textos_df.head(3)

,tit,tex,tex_en,tok_en
0,"Cómo despedirse de alguien, a quien ya no salu...",A Patricia Primero: Dibújate un interrogante ...,A Patricia first: Draw a question on the lips ...,patricia draw question lip match white disbeli...
1,Números rojos,Números rojos Subí la apuesta a que te bajaba...,Red numbers I went up the bet that lowered you...,red number go bet lower pant end buy battery c...
2,En espiral,"Sufro una crisis de infertilidad, y ya no sé c...","I suffer a crisis of infertility, and I do not...",suffer crisis infertility know lethargy mascot...


In [203]:
textos_df["tok_en"] = textos_df.tex_en.apply(tokenizer)

In [206]:
textos_df.head(3)

,tit,tex,tex_en,tok_en
0,"Cómo despedirse de alguien, a quien ya no salu...",A Patricia Primero: Dibújate un interrogante ...,A Patricia first: Draw a question on the lips ...,patricia draw question lip match white disbeli...
1,Números rojos,Números rojos Subí la apuesta a que te bajaba...,Red numbers I went up the bet that lowered you...,red number go bet lower pant end buy battery c...
2,En espiral,"Sufro una crisis de infertilidad, y ya no sé c...","I suffer a crisis of infertility, and I do not...",suffer crisis infertility know lethargy mascot...


In [204]:
textos_en = pd.DataFrame(textos_df)

In [205]:
textos_en.head(3)

,tit,tex,tex_en,tok_en
0,"Cómo despedirse de alguien, a quien ya no salu...",A Patricia Primero: Dibújate un interrogante ...,A Patricia first: Draw a question on the lips ...,patricia draw question lip match white disbeli...
1,Números rojos,Números rojos Subí la apuesta a que te bajaba...,Red numbers I went up the bet that lowered you...,red number go bet lower pant end buy battery c...
2,En espiral,"Sufro una crisis de infertilidad, y ya no sé c...","I suffer a crisis of infertility, and I do not...",suffer crisis infertility know lethargy mascot...


In [186]:
#textos_en.to_csv("data/text_en.csv")

In [207]:
def blobpol(col):
    total = []
    
    blob = TextBlob(col)
    total.append(blob.sentiment[0])
    total.append(blob.sentiment[1])
    
    sia = SentimentIntensityAnalyzer()
    polaridad = sia.polarity_scores(col)
    total.append(polaridad["neg"])
    total.append(polaridad["neu"])
    total.append(polaridad["pos"])
    total.append(polaridad["compound"])
    return total

In [208]:
textos_en["sentiments"] = textos_en.tok_en.apply(blobpol)

In [209]:
textos_en.head(3)

,tit,tex,tex_en,tok_en,sentiments
0,"Cómo despedirse de alguien, a quien ya no salu...",A Patricia Primero: Dibújate un interrogante ...,A Patricia first: Draw a question on the lips ...,patricia draw question lip match white disbeli...,"[0.04540816326530611, 0.3739795918367347, 0.23..."
1,Números rojos,Números rojos Subí la apuesta a que te bajaba...,Red numbers I went up the bet that lowered you...,red number go bet lower pant end buy battery c...,"[0.09999999999999999, 0.38571428571428573, 0.1..."
2,En espiral,"Sufro una crisis de infertilidad, y ya no sé c...","I suffer a crisis of infertility, and I do not...",suffer crisis infertility know lethargy mascot...,"[-0.03958333333333334, 0.6124999999999999, 0.2..."


In [210]:
text_sent = textos_en
#text_sent.to_csv("data/text_sent.csv")

In [211]:
text_sent.head(3)

,tit,tex,tex_en,tok_en,sentiments
0,"Cómo despedirse de alguien, a quien ya no salu...",A Patricia Primero: Dibújate un interrogante ...,A Patricia first: Draw a question on the lips ...,patricia draw question lip match white disbeli...,"[0.04540816326530611, 0.3739795918367347, 0.23..."
1,Números rojos,Números rojos Subí la apuesta a que te bajaba...,Red numbers I went up the bet that lowered you...,red number go bet lower pant end buy battery c...,"[0.09999999999999999, 0.38571428571428573, 0.1..."
2,En espiral,"Sufro una crisis de infertilidad, y ya no sé c...","I suffer a crisis of infertility, and I do not...",suffer crisis infertility know lethargy mascot...,"[-0.03958333333333334, 0.6124999999999999, 0.2..."


In [100]:
#clima_df[['Tmax','Tavg','Tmin']] = clima_df["Temperature (° F)"].str.split(" ",expand=True)

In [184]:
#text_sent[["blobpol","blobsub","sianeg","sianeu","siapos","siacomp"]] = [sent[0] for sent in text_sent["sentiments"] for s in sent]

In [183]:
# for sent in text_sent["sentiments"]:
#     print(sent[0])
# #     for s in sent:
# #         print(s)

In [177]:
def sentimentsdiv():
    for sent in text_sent["sentiments"]:
        text_sent["blobpol"] = sent[0]
        text_sent["blobsub"] = sent[1]
        text_sent["sianeg"] = sent[2]
        text_sent["sianeu"] = sent[3]
        text_sent["siapos"] = sent[4]
        text_sent["siacomp"] = sent[5]
    return 

[0.04540816326530611, 0.3739795918367347, 0.234, 0.582, 0.185, -0.8454]
[0.09999999999999999, 0.38571428571428573, 0.107, 0.556, 0.338, 0.9393]
[-0.03958333333333334, 0.6124999999999999, 0.293, 0.56, 0.147, -0.872]
[-0.22999999999999998, 0.6399999999999999, 0.29, 0.572, 0.138, -0.802]
[-0.16666666666666666, 0.5555555555555555, 0.099, 0.699, 0.202, 0.4215]
[-0.009166666666666662, 0.3391666666666667, 0.277, 0.525, 0.198, -0.4019]
[-0.1134920634920635, 0.5566137566137567, 0.288, 0.474, 0.237, -0.4767]
[-0.22142857142857145, 0.4386904761904762, 0.153, 0.757, 0.089, -0.5106]
[-0.6333333333333333, 0.95, 0.12, 0.583, 0.297, 0.8519]
[0.06863636363636363, 0.29958333333333337, 0.153, 0.665, 0.182, 0.6124]
[0.4, 0.45, 0.162, 0.838, 0.0, -0.4019]
[-0.1714285714285714, 0.41904761904761906, 0.188, 0.682, 0.129, -0.5423]
[0.3666666666666667, 0.4666666666666666, 0.0, 0.603, 0.397, 0.9153]
[-0.06375, 0.22666666666666666, 0.068, 0.817, 0.115, 0.7037]
[0.2374183006535948, 0.5259259259259259, 0.121, 0.592

In [191]:
text_sent.drop("blobpol", axis=1, inplace = True)

In [153]:
#text_sent["blobpol"] = pd.Series()
for i, row in text_sent.iterrows():
    text_sent.loc[i]["blobpol"] = text_sent["sentiments"].values[i][0]

/home/sinatxe/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [192]:
text_sent

,tit,tex,tex_en,tok_en,sentiments
0,"Cómo despedirse de alguien, a quien ya no salu...",A Patricia Primero: Dibújate un interrogante ...,A Patricia first: Draw a question on the lips ...,patricia draw question lip match white disbeli...,"[0.04540816326530611, 0.3739795918367347, 0.23..."
1,Números rojos,Números rojos Subí la apuesta a que te bajaba...,Red numbers I went up the bet that lowered you...,red number go bet lower pant end buy battery c...,"[0.09999999999999999, 0.38571428571428573, 0.1..."
2,En espiral,"Sufro una crisis de infertilidad, y ya no sé c...","I suffer a crisis of infertility, and I do not...",suffer crisis infertility know lethargy mascot...,"[-0.03958333333333334, 0.6124999999999999, 0.2..."
3,En mis recuerdos,"En mis recuerdos siempre hay un cenicero, ropa...","In my memories there is always an ashtray, dir...",memory ashtray dirty clothe floor clock lose t...,"[-0.22999999999999998, 0.6399999999999999, 0.2..."
4,Aquel rojo,Me he vuelto a malpintar las uñas de aquel roj...,I have returned to mischiece the nails of that...,return mischiece nail red combine underwear fi...,"[-0.16666666666666666, 0.5555555555555555, 0.0..."
5,Notas al margen,"A veces me enamoro, otras abro los ojos. No si...","Sometimes I fall in love, others open my eyes....",fall love open eye write like notebook coffee ...,"[-0.009166666666666662, 0.3391666666666667, 0...."
6,Carnaval de musas,"Estas musas que gustan de verme hundida, traen...","These muses that like to see me sunk, bring to...",muse like sink bring doubt certainty certainly...,"[-0.1134920634920635, 0.5566137566137567, 0.28..."
7,Su mente rota,"Los pasillos eran cubos y las vistas, líneas r...","The hallways were cubes and views, straight li...",hallway cube view straight line symmetry stree...,"[-0.22142857142857145, 0.4386904761904762, 0.1..."
8,Y otra vez...,"Creía que tenía el tiempo en mis manos, y de m...","He thought he had time in my hands, and from m...",think time hand finger goodbye escape opportun...,"[-0.6333333333333333, 0.95, 0.12, 0.583, 0.297..."
9,Y otros destinos escritos,Me he acordado de la última vez que no me enam...,I have remembered the last time I did not fall...,remember time fall love liver break atillo sto...,"[0.06863636363636363, 0.29958333333333337, 0.1..."
